In [30]:
import pdftotext
import pandas as pd
import numpy as np
import re
import os

#%% # snetiment analysis
import numpy as np
import pandas as pd

import re
import string 

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

import nltk 
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords          # module for stop words that come with NLTK
nltk.download('stopwords')
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/timliu/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/timliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define function for "Cleaning" and "participants list"

In [10]:
def cleaning_text(contents):
    # change the text input to df
    df = pd.DataFrame(contents)
    # remove the unnessary string
    df[0] = df[0].str.replace('\n','')
    df[0] = df[0].str.replace('Bloomberg Transcript','')
    df[0] = df[0].str.replace('\x0c\n','')
    df[0] = df[0].str.replace('FINAL','')
    df[0] = df[0].str.replace('A - ','')
    df[0] = df[0].str.replace('Q - ','')
    # using re to remove the unnessary string
    def drop_unnessary(x):
        page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
        BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
        Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
        Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
        Date = re.findall(r'Date:', x) # Date: 2015-03-10
        if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_unnessary(x))
    df = df[true_false]

    # drop the final page declaration
    df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
    df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
    df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
    df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
    df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
    df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
    df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
    df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
    df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
    # df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  # we will need this to identify the last participant
    df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
    # ¬© could not be identified, would apply re
    def drop_Bloomberg_mark(x):
        Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
        if Bloomberg_mark == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
    df = df[true_false]

    # drop the empthy row
    df = df[df[0] != '']
    df = df[df[0] != '']

    return df

def participants_list(df):
    # reset the index to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    #  'Company Participants' index
    Participant_start_index = df.index[df.iloc[:,0] == 'Company Participants'].tolist()
    #  'Other Participants' index
    Participant_middle_index = df.index[df.iloc[:,0] == 'Other Participants'].tolist()
    #  'MANAGEMENT DISCUSSION SECTION' index, is the beginning of the management discussion, would stop before this row
    Participant_end_index = df.index[df.iloc[:,0] == 'MANAGEMENT DISCUSSION SECTION' ].tolist()
    # try to find the 'MANAGEMENT DISCUSSION SECTION' or 'Presentation' index
    if Participant_end_index == []:
        Participant_end_index = df.index[df.iloc[:,0] == 'Presentation'].tolist()
    print(Participant_start_index, Participant_middle_index, Participant_end_index)

    # make the list of company_paticipants 
    company_paticipants = df.loc[Participant_start_index[0]+1:Participant_middle_index[0]-1]
    company_paticipants.drop(company_paticipants.index[company_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    company_paticipants = company_paticipants.values.tolist()
    # and other_participants
    other_paticipants = df.loc[Participant_middle_index[0]+1:Participant_end_index[0]-1]
    other_paticipants.drop(other_paticipants.index[other_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    other_paticipants = other_paticipants.values.tolist()

    # after extract the paticipants, we can drop those information to make the transcript more clear
    df = df.reset_index(drop=True)
    df = df.drop(range(df.index[df.iloc[:,0] == 'Company Participants'].tolist()[0],df.index[df.iloc[:,0].isin(['MANAGEMENT DISCUSSION SECTION','Presentation'])].tolist()[0]+1))
    # drop the first row of the df
    df = df.reset_index(drop=True)
    df = df.iloc[1: , :]
    # reset the index again to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    # # save to csv
    # df.to_csv('/Users/timliu/Desktop/output/df.csv')
    return df, company_paticipants, other_paticipants

In [11]:
### Testing on the single company
path = "/Users/timliu/Documents/GitHub/data_collecting/BBG_original_file/European (Re)Insurers/HNR1 GY" #資料夾目錄
save_path = "/Users/timliu/Documents/GitHub/data_collecting/output/HNR1 GY_text"

In [12]:
df = pd.DataFrame()
# create a dataframe with 2500 rows (to make sure all the paragraphs are including)
df_clean_na = pd.DataFrame(np.zeros((2500,1)), columns=['index'])
all_participants = []
company_paticipants_list = []
other_paticipants_list = []

files= os.listdir(path) #得到資料夾下的所有檔名稱
for file in files:
    if file.endswith(".pdf"):
        # print(file)
        # Load PDF
        with open(path+"/"+file, "rb") as f:
            pdf = pdftotext.PDF(f)
        # Save all text to a txt file.
        with open(save_path+"/"+file.replace(".pdf", ".txt"), "w") as f:
            f.write("\n\n".join(pdf))
        # open the text file
        with open(save_path+"/"+file.replace(".pdf", ".txt")) as f:
            contents = f.readlines()
            df_clean = cleaning_text(contents)
            # extract all the participants
            df_pure_text,company_paticipants,other_paticipants = participants_list(df_clean)
            all_participants.append(company_paticipants)
            company_paticipants_list.append(other_paticipants)
            other_paticipants_list.append(company_paticipants)
            # using the file name to set as the dataframe's column name
            # df[f"{files.index(file)}"] = df_clean
            df[f"{files[files.index(file)]}"] = df_pure_text
            df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)

# drop the first column of the df
df_clean_na = df_clean_na.iloc[:,1:]
df_clean_na

[1] [7] [14]


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


[1] [10] [24]
[1] [4] [12]
[1] [4] [17]
[1] [4] [17]
[1] [5] [21]
[1] [6] [13]
[1] [5] [12]
[1] [4] [17]
[1] [4] [13]
[1] [6] [13]
[1] [8] [21]
[1] [4] [16]
[1] [6] [20]
[1] [5] [16]
[1] [6] [18]
[1] [6] [14]
[1] [11] [27]
[1] [5] [19]
[1] [5] [15]
[1] [6] [16]
[1] [4] [17]
[1] [6] [15]
[1] [4] [12]
[1] [5] [12]
[1] [4] [12]
[2] [5] [14]
[1] [4] [13]
[1] [4] [12]
[1] [10] [28]
[1] [6] [14]
[2] [5] [13]
[1] [4] [15]
[1] [4] [17]
[1] [5] [15]
[1] [5] [10]
[1] [9] [17]


,20211104_Hannover_Rueck_SE-_Earnings_Call_2021-11-4_RT000000002967437630.pdf,20191023_Hannover_Rueck_SE-_Shareholder_Mtg_Call_2019-10-23_SD000000002903050937.pdf,20150506_Hannover_Rueck_SE-_Earnings_Call_2015-5-6_FS000000002212304783.pdf,20160804_Hannover_Rueck_SE-_Earnings_Call_2016-8-4_SD000000002853744569.pdf,20171108_Hannover_Rueck_SE-_Earnings_Call_2017-11-8_SD000000002868833083.pdf,20160310_Hannover_Rueck_SE-_Earnings_Call_2016-3-10_FS000000002259507768.pdf,20201104_Hannover_Rueck_SE-_Earnings_Call_2020-11-4_RT000000002931797209.pdf,20210204_Hannover_Rueck_SE-_M-A_Call_2021-2-4_RT000000002949284264.pdf,20180809_Hannover_Rueck_SE-_Earnings_Call_2018-8-9_SD000000002876144587.pdf,20190507_Hannover_Rueck_SE-_Earnings_Call_2019-5-7_DN000000002633135788.pdf,...,20200205_Hannover_Rueck_SE-_M-A_Call_2020-2-5_DN000000002787035776.pdf,20160510_Hannover_Rueck_SE-_Earnings_Call_2016-5-10_FS000000002275763746.pdf,20161020_Hannover_Rueck_SE-_Guidance_Call_2016-10-20_SD000000002902464788.pdf,20210505_Hannover_Rueck_SE-_Earnings_Call_2021-5-5_DN000000002956339792.pdf,20190205_Hannover_Rueck_SE-_Guidance_Call_2019-2-5_SD000000002901846468.pdf,20181108_Hannover_Rueck_SE-_Earnings_Call_2018-11-8_SD000000002879406671.pdf,20150805_Hannover_Rueck_SE-_Earnings_Call_2015-8-5_FS000000002223534191.pdf,20190307_Hannover_Rueck_SE-_Earnings_Call_2019-3-7_DN000000002597819789.pdf,20200506_Hannover_Rueck_SE-_Earnings_Call_2020-5-6_DN000000002833326951.pdf,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Call_2021-10-14_SD000000002965861183.pdf
0,Operator,Karl Steinle,Operator,Operator,Ulrich Wallin,Karl Steinle,Operator,Operator,Ulrich Wallin,Operator,...,Operator,Operator,Karl Steinle,Operator,Call,Ulrich Wallin,Operator,Operator,Operator,Karl Steinle
1,"Good morning, ladies and gentlemen. I welcome ...",Well. Good morning to all of you. Welcome to H...,"Good morning, ladies and gentlemen, and welcom...","Good morning, ladies and gentlemen. Welcome to...","Good morning, ladies and gentlemen. I'd like t...","Good afternoon everybody here in Frankfurt, an...","Good morning, ladies and gentlemen. I welcome ...","Good morning, ladies and gentlemen. I welcome ...","Yes. Good morning, ladies and gentlemen. I'd l...","Good morning, ladies and gentlemen. I welcome ...",...,Good morning ladies and gentlemen. I welcome y...,"Good morning, ladies and gentlemen, and welcom...","Good morning, to all of you. Welcome to Hannov...","Good morning, ladies and gentlemen. I welcome ...",Operator,"Good morning, ladies and gentlemen. I'd like t...","Good morning, ladies and gentlemen. I welcome ...","Well, good afternoon to everybody here in Lond...","Good morning, ladies and gentlemen. I welcome ...","Hello. Good morning, to the Hannover Re's Inve..."
2,Conference Call on the Q3 2021 Financial Resul...,really delighted that so many of you were able...,Conference Call on Interim Results 1/2015. For...,conference call on interim results (technical ...,presenting the results for the first nine mont...,via the Internet. Welcome to Hannover Re's Ana...,conference call on the Q3 2020 results. For yo...,"Conference Call on 1st January, 2021, Property...",presenting the results for the first half year...,Conference Call on Q1 2019 Results. For your i...,...,"Conference Call on January 1, 2020 Property an...",Conference Call on Interim Results Q1 2016. Fo...,Karl Steinle. And I'm really delighted that so...,Call on the Q1 2021 Results. For your informat...,"Good morning, ladies and gentlemen. I welcome ...",presenting our results for the first nine mont...,Conference Call on Interim Results Q2 2015. Fo...,Internet. Welcome to Hannover Re's Analyst Con...,Conference Call on Q1 2020 Financial Results. ...,on behalf of the entire management team. Again...
3,"being recorded. At this time, I would like to ...","Steinle. And I'm, among other things, responsi...","recorded. At this time, I would like to hand t...","is being recorded. At this time, I would like ...",Roland Vogel.,see that so

In [14]:
def clean_participants_list(def_participants):
    # get the value inside the def_participants 
    def_participants = [item for sublist in def_participants for item in sublist]
    def_participants = [i[0] for i in def_participants]
    # print(def_participants)
    # %%
    # exclude the title of the participants, i.e.'Roland Vogel, CFO' to 'Roland Vogel" by using re
    def_participants = [re.sub(r'\,.*', '', participant) for participant in def_participants]
    # exclude the 'Property & Casualty Reinsurance'
    def_participants = [re.sub(r'Property & Casualty Reinsurance', '', participant) for participant in def_participants]
    # exclude the '[0682QB-E Ulrich Wallin]'
    def_participants = [re.sub(r'\[0682QB-E Ulrich Wallin\]', '', participant) for participant in def_participants]
    # drop duplicated participants
    # def_participants = [i[0] for i in def_participants]
    # drop the empty string
    def_participants = [participant for participant in def_participants if participant != '']
    # remove the sapce in the string
    def_participants = [participant.strip() for participant in def_participants]
    # add the 'Operator' to the list
    def_participants.append('Operator')

    # drop the duplicated participants
    def_participants_copy = def_participants.copy()
    def_participants = []
    # drop the duplicated participants
    for i in def_participants_copy: 
        if i not in def_participants: 
            def_participants.append(i) 
    def_participants = sorted(def_participants)
    
    return def_participants

all_participants = clean_participants_list(all_participants)
company_paticipants_list = clean_participants_list(company_paticipants_list)
other_paticipants_list = clean_participants_list(other_paticipants_list)


In [19]:
concat_df = pd.DataFrame()
for column in df_clean_na.columns:
    # identify all the rows in df with all_participants in it
    both_participants_row_index = df_clean_na[df_clean_na[column].isin(all_participants)].index.tolist()
    # apply the both_participants_row_index to the df_clean_na['participants']
    concat_df[column] = df_clean_na[column]
    concat_df[f"participants_{column}"] = df_clean_na[column].apply(lambda x: x if x in all_participants else np.nan)
    # fill the NaN with the value of the previous row
    concat_df[f"participants_{column}"] = concat_df[f"participants_{column}"].fillna(method='ffill')

,20211104_Hannover_Rueck_SE-_Earnings_Call_2021-11-4_RT000000002967437630.pdf,participants_20211104_Hannover_Rueck_SE-_Earnings_Call_2021-11-4_RT000000002967437630.pdf,20191023_Hannover_Rueck_SE-_Shareholder_Mtg_Call_2019-10-23_SD000000002903050937.pdf,participants_20191023_Hannover_Rueck_SE-_Shareholder_Mtg_Call_2019-10-23_SD000000002903050937.pdf,20150506_Hannover_Rueck_SE-_Earnings_Call_2015-5-6_FS000000002212304783.pdf,participants_20150506_Hannover_Rueck_SE-_Earnings_Call_2015-5-6_FS000000002212304783.pdf,20160804_Hannover_Rueck_SE-_Earnings_Call_2016-8-4_SD000000002853744569.pdf,participants_20160804_Hannover_Rueck_SE-_Earnings_Call_2016-8-4_SD000000002853744569.pdf,20171108_Hannover_Rueck_SE-_Earnings_Call_2017-11-8_SD000000002868833083.pdf,participants_20171108_Hannover_Rueck_SE-_Earnings_Call_2017-11-8_SD000000002868833083.pdf,...,20181108_Hannover_Rueck_SE-_Earnings_Call_2018-11-8_SD000000002879406671.pdf,participants_20181108_Hannover_Rueck_SE-_Earnings_Call_2018-11-8_SD000000002879406671.pdf,20150805_Hannover_Rueck_SE-_Earnings_Call_2015-8-5_FS000000002223534191.pdf,participants_20150805_Hannover_Rueck_SE-_Earnings_Call_2015-8-5_FS000000002223534191.pdf,20190307_Hannover_Rueck_SE-_Earnings_Call_2019-3-7_DN000000002597819789.pdf,participants_20190307_Hannover_Rueck_SE-_Earnings_Call_2019-3-7_DN000000002597819789.pdf,20200506_Hannover_Rueck_SE-_Earnings_Call_2020-5-6_DN000000002833326951.pdf,participants_20200506_Hannover_Rueck_SE-_Earnings_Call_2020-5-6_DN000000002833326951.pdf,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Call_2021-10-14_SD000000002965861183.pdf,participants_20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Call_2021-10-14_SD000000002965861183.pdf
0,Operator,Operator,Karl Steinle,Karl Steinle,Operator,Operator,Operator,Operator,Ulrich Wallin,Ulrich Wallin,...,Ulrich Wallin,Ulrich Wallin,Operator,Operator,Operator,Operator,Operator,Operator,Karl Steinle,Karl Steinle
1,"Good morning, ladies and gentlemen. I welcome ...",Operator,Well. Good morning to all of you. Welcome to H...,Karl Steinle,"Good morning, ladies and gentlemen, and welcom...",Operator,"Good morning, ladies and gentlemen. Welcome to...",Operator,"Good morning, ladies and gentlemen. I'd like t...",Ulrich Wallin,...,"Good morning, ladies and gentlemen. I'd like t...",Ulrich Wallin,"Good morning, ladies and gentlemen. I welcome ...",Operator,"Well, good afternoon to everybody here in Lond...",Operator,"Good morning, ladies and gentlemen. I welcome ...",Operator,"Hello. Good morning, to the Hannover Re's Inve...",Karl Steinle
2,Conference Call on the Q3 2021 Financial Resul...,Operator,really delighted that so many of you were able...,Karl Steinle,Conference Call on Interim Results 1/2015. For...,Operator,conference call on interim results (technical ...,Operator,presenting the results for the first nine mont...,Ulrich Wallin,...,presenting our results for the first nine mont...,Ulrich Wallin,Conference Call on Interim Results Q2 2015. Fo...,Operator,Internet. Welcome to Hannover Re's Analyst Con...,Operator,Conference Call on Q1 2020 Financial Results. ...,Operator,on behalf of the entire management team. Again...,Karl Steinle
3,"being recorded. At this time, I would like to ...",Operator,"Steinle. And I'm, among other things, responsi...",Karl Steinle,"recorded. At this time, I would like to hand t...",Operator,"is being recorded. At this time, I would like ...",Operator,Roland Vogel.,Ulrich Wallin,...,Roland Vogel.,Ulrich Wallin,being recorded.,Operator,truly a pleasure to see so many of you taking ...,Operator,"being recorded. At this time, I would like to ...",Operator,pandemic. So we are not broadcasting from Cope...,Karl Steinle
4,Jean-Jacques Henchoz,Jean-Jacques Henchoz,comms.,Karl Steinle,"Wallin, Chief Executive Officer. Please go ahe...",Operator,"Ulrich Wallin, Chief Executive Officer. Please...",Operator,"After years of moderate losses, we saw an accu...",Ulrich Wallin,...,The most significant event that had an influen...,Ulrich Wall

In [51]:
tmp_df['company_name'] = tmp_df.columns.to_list()[1]
tmp_df

,paragraph,participants,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name
1,"Hello. Good morning, to the Hannover Re's Inve...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
2,on behalf of the entire management team. Again...,Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
3,pandemic. So we are not broadcasting from Cope...,Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
4,Hanover. I'm happy that so many of you are alr...,Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
5,"24th edition, and therefore, a long-standing t...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
...,...,...,...,...,...,...,...
770,"Our ordinary income, that's very pleasing. And...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
771,portfolio has been very reliable in terms of p...,Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
772,"low interest rate environment, of course is ch...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants
773,"inflation risk side, again I think it's over -...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,participants


In [62]:
no_par_df = pd.DataFrame()
# identify the len before NaN of each column
for column in df_clean_na.columns:
    # exclude the row if no_par_df[column]==no_par_df[f"participants_{column}"]
    no_par_df = concat_df[concat_df[column] != concat_df[f"participants_{column}"]]

# use len(no_par_df.columns.to_list()) to write a for loop
model_df = pd.DataFrame()
for i in range(int(len(no_par_df.columns.to_list())/2)):
    tmp_df = pd.DataFrame()
    tmp_df = no_par_df.iloc[:,(i*2):(i*2)+2]
    # extract the index as column from the text
    tmp_df['file_name'] = tmp_df.columns.to_list()[0]
    # extract the date from the index column
    tmp_df['date'] = tmp_df['file_name'].apply(lambda x: x.split('_')[0])
    # change the date column to datetime
    tmp_df['date'] = pd.to_datetime(tmp_df['date'])
    # rename to be consistent with the column name
    tmp_df.columns = ["paragraph", "participants", "file_name","date"]
    # if the 'participants' column's value equals to any of the company_paticipants_list, other_paticipants_list, then set the value to 0
    tmp_df['company_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in company_paticipants_list else 0)
    tmp_df['other_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in other_paticipants_list else 0)
    # drop the row if the column "paragraph" is NaN
    tmp_df = tmp_df.dropna(subset=['paragraph'], how='all')
    tmp_df['company_name1']  = tmp_df['file_name'].apply(lambda x: x.split('_')[1])
    tmp_df['company_name2']  = tmp_df['file_name'].apply(lambda x: x.split('_')[2])
    tmp_df['company_name'] = tmp_df["company_name1"] + " " + tmp_df["company_name2"]
    # drop the 'company_name1' and 'company_name2' column
    tmp_df = tmp_df.drop(columns=['company_name1', 'company_name2'])
    model_df = model_df.append(tmp_df)

model_df

/var/folders/sq/sxhsy98j6f57m406rd5lz1_r0000gn/T/ipykernel_34116/2318456439.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['file_name'] = tmp_df.columns.to_list()[0]
/var/folders/sq/sxhsy98j6f57m406rd5lz1_r0000gn/T/ipykernel_34116/2318456439.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['date'] = tmp_df['file_name'].apply(lambda x: x.split('_')[0])
/var/folders/sq/sxhsy98j6f57m406rd5lz1_r0000gn/T/ipykernel_34116/2318456439.py:17: SettingWithCopyWarning: 
A value is trying to be se

,paragraph,participants,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name
1,"Good morning, ladies and gentlemen. I welcome ...",Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1,Hannover Rueck
2,Conference Call on the Q3 2021 Financial Resul...,Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1,Hannover Rueck
3,"being recorded. At this time, I would like to ...",Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1,Hannover Rueck
4,Jean-Jacques Henchoz,Jean-Jacques Henchoz,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,0,1,Hannover Rueck
5,"Well, good morning, everyone and welcome to ou...",Jean-Jacques Henchoz,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,0,1,Hannover Rueck
...,...,...,...,...,...,...,...
770,"Our ordinary income, that's very pleasing. And...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck
771,portfolio has been very reliable in terms of p...,Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck
772,"low interest rate environment, of course is ch...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck
773,"inflation risk side, again I think it's over -...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck


# Sentiment analysis

In [63]:
# write a function to split the text by '.' in paragraph_split_df
p_to_s_split_df = model_df[['paragraph','participants']].copy()

def split_text(text):
    text = text.split(". ")
    return text
# apply the function to the paragraph_split_df
p_to_s_split_df['paragraph'] = p_to_s_split_df['paragraph'].apply(lambda x: split_text(x))

# get the df split by sentence
sentence_split_df = pd.DataFrame()
for i in range(len(p_to_s_split_df)):
    sentence_list = p_to_s_split_df['paragraph'].iloc[i]
    sentence_split_single_df = pd.DataFrame (sentence_list, columns = ['sentence'])
    sentence_split_single_df['participants'] = p_to_s_split_df['participants'].iloc[i]
    sentence_split_single_df['paragraph_num'] = i
    sentence_split_df = sentence_split_df.append(sentence_split_single_df, ignore_index=True)
# drop if the 'sentence' is empty
sentence_split_df = sentence_split_df.dropna(inplace=False)

sentence_split_df

,sentence,participants,paragraph_num
0,"Good morning, ladies and gentlemen",Operator,0
1,I welcome you to today's Hannover Re Internati...,Operator,0
2,Conference Call on the Q3 2021 Financial Results,Operator,1
3,"For your information, this conference is",Operator,1
4,being recorded,Operator,2
...,...,...,...
38575,On the,Karl Steinle,25616
38576,"inflation risk side, again I think it's over -...",Karl Steinle,25617
38577,Inflation drivers are monitored,Karl Steinle,25618
38578,We are monitoring them very closely,Karl Steinle,25618


In [64]:
# documents
docs_negative = [(t, "neg") for t in twitter_samples.strings("negative_tweets.json")]
docs_positive = [(t, "pos") for t in twitter_samples.strings("positive_tweets.json")]
print("==========================================================")
print(f'There are {len(docs_negative)} negative sentences.')
print(f'There are {len(docs_positive)} positive sentences.')

# spliting dataset 
train_set = docs_negative[:3500] + docs_positive[:3500]
test_set = docs_negative[3500:4250] + docs_positive[3500:4250]
valid_set = docs_negative[4250:] + docs_positive[4250:]

# clean text
def process_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    #text = text.str
    text = str(text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'#', '', text)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    text_clean = []
    for word in text_tokens:
        if (word not in stopwords_english and  
                word not in string.punctuation): 
            stem_word = stemmer.stem(word)  # stemming word
            text_clean.append(stem_word)
            
    sentence = ' '.join(text_clean)
    
    return sentence

# categorical label
def cat_label(label):
    if label == 'neg':
        value = -1
    elif label == 'pos':
        value = 1
    return value 

# split for x and y 
def xy(dataset):
    df = pd.DataFrame(dataset, columns = ['text', 'label'])
    df['text_clean'] = df['text'].apply(lambda r: process_text(r))
    #df['categorical_label'] = df.label.factorize()[0]
    df['categorical_label'] = df['label'].apply(lambda r: cat_label(r))

    x = df.text_clean
    y = df.categorical_label

    return x, y

# dataframe
x_train, y_train = xy(train_set)
x_test, y_test = xy(test_set)
x_valid, y_valid = xy(valid_set)

## using the naive bayes classifier
model = Pipeline([
    ('bow',CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print("==========================================================")
print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))
print(accuracy_score(y_pred,y_test))

There are 5000 negative sentences.
There are 5000 positive sentences.
[[593 235]
 [157 515]]
              precision    recall  f1-score   support

          -1       0.79      0.72      0.75       828
           1       0.69      0.77      0.72       672

    accuracy                           0.74      1500
   macro avg       0.74      0.74      0.74      1500
weighted avg       0.74      0.74      0.74      1500

0.7386666666666667


In [65]:
#take a copy
length_sentence = sentence_split_df.copy()
# calculate length of each sentence
length_sentence['length_of_sentence'] = length_sentence['sentence'].apply(lambda r: len(r))
# calculate length by paragraph_num
length_para = length_sentence.groupby(['paragraph_num']).sum().reset_index(drop=False)
length_para.rename(columns={'length_of_sentence': 'length_of_para'}, inplace=True)
# merged into 1 datafram
cal_sentiment = pd.merge(length_sentence, length_para, on=["paragraph_num"])
cal_sentiment['sentiment_score']=df_sentence['sentiment_score']
# calculate weighted 
cal_sentiment['sentiment_by_weighted']=cal_sentiment['length_of_sentence']/cal_sentiment['length_of_para']*cal_sentiment['sentiment_score']
# append back to the model_df
score_df = cal_sentiment[['paragraph_num','sentiment_by_weighted']]
score_list = np.array(score_df.groupby(['paragraph_num']).sum().reset_index(drop=True).values)
# add the score_list into paragraph_split_df['sentiment_score']
model_df = model_df.assign(sentiment_score=score_list)
model_df


,paragraph,participants,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name,sentiment_score
1,"Good morning, ladies and gentlemen. I welcome ...",Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1,Hannover Rueck,1.000000
2,Conference Call on the Q3 2021 Financial Resul...,Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1,Hannover Rueck,-1.000000
3,"being recorded. At this time, I would like to ...",Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1,Hannover Rueck,-0.814570
4,Jean-Jacques Henchoz,Jean-Jacques Henchoz,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,0,1,Hannover Rueck,-1.000000
5,"Well, good morning, everyone and welcome to ou...",Jean-Jacques Henchoz,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,0,1,Hannover Rueck,1.000000
...,...,...,...,...,...,...,...,...
770,"Our ordinary income, that's very pleasing. And...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck,1.000000
771,portfolio has been very reliable in terms of p...,Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck,0.555556
772,"low interest rate environment, of course is ch...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck,0.857143
773,"inflation risk side, again I think it's over -...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1,Hannover Rueck,-1.000000


# TP modelling